In [5]:
# Small Language Model(SLLM) 

# Step 02 - 트랜스포머 허깅페이스 데이터셋(WikiText-2)을 적용하여 Small Language Model(SLLM) 구축 및 텍스트 생성
# 데이터셋 -> 데이터로더 -> GPU 설정 -> SLLM 모델 정의 -> 학습 -> 텍스트 생성 -> 테스트

# 핵심 문제 - 영어 기반이라 감성적인 한국어 여행 블로그를 생성하기엔 한계
# - 단어 반복 (“jenkin”) : WikiText-2에 자주 등장하는 고유명사
# - 문맥 불일치 : 프롬프트는 감성 여행, 데이터는 역사/스포츠
# - 의미 연결 부족 : 모델이 감성적 흐름을 학습하지 못함

# 기능 추가
# - 샘플링 방식(top-k, top-p, temperature) 적용: 다양성 확보
# - 프롬프트 튜닝: 감성적 블로그 스타일 유도
# - 후처리(clean_text): 출력 품질 향상

In [5]:
# 1) import 및 GPU 설정
# - torch, nn, Dataset, DataLoader, load_dataset 등 필요한 모듈 사용
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset # HuggingFace Datasets
from transformers import AutoTokenizer

# - GPU 사용 여부 : device 설정은 GPU가 있을 경우 자동으로 CUDA를 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Pytorch Version : {torch.__version__}, Device : {device}')

# - AutoTokenizer 사용(bert-base-uncased)
auto_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Pytorch Version : 2.2.2, Device : cpu


In [ ]:
# 2) WikiText-2 데이터셋 로딩 및 전처리
# - Hugging Face에서 WikiText-2 로드 및 다운로드
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
# print('dataset : ', dataset)
# dataset :  Dataset({
#     features: ['text'],
#     num_rows: 36718
# })

# 간단한 토크나이저 사용 - 단어 단위
# from collections import Counter

# - 단어 단위로 토큰화 함수
def tokenizer(text):
    # return text.lower().split()
    return auto_tokenizer.tokenize(text)

# 전체 텍스트 토큰화
tokens = []
for item in dataset:
    tokens.extend(tokenizer(item['text']))

# 단어 사전 생성
# vocab = list(set(tokens))
# - 코드수정 이유 : Transformer 모델은 word2idx를 통해 단어를 숫자로 매핑하고, 이 숫자가 Embedding Layer의 인덱스로 사용
# - sorted(set(tokens))을 사용하면 항상 같은 단어 → 인덱스 매핑이 되므로, 모델의 학습과 추론 결과를 일관되게 재현 할 수 있음,
vocab = sorted(set(tokens))
word2idx = { word:idx for idx, word in enumerate(vocab) } # 단어 리스트 -> 딕셔너리(value:key), index 값 출력
idx2word = { idx:word for word, idx in word2idx.items() } # 딕셔너리(key:value), word value 값 출력

# 시퀀스 생성(입력: [32 단어], 출력: [32 단어] 등)
sequence_length = 32
data = []
for i in range(len(tokens) - sequence_length):
    input_seq = tokens[i:i+sequence_length]
    target_seq = tokens[i+1:i+sequence_length+1]

    # word2idx[word] 딕셔너리 index 값을 추출 -> 텐서 데이터로 변환
    data.append((
        torch.tensor([ word2idx[word] for word in input_seq ]),
        torch.tensor([ word2idx[word] for word in target_seq ]) 
    ))

2303663


In [41]:
# 3) 데이터셋 및 데이터로더 생성
class WikiTextDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# - 코드 수정 : 데이터 길이에 따라 최대 40000개 까지 슬라이싱 처리(데이터 부족해도 안전하게 처리, 데이터 크기에 따라 자동 조절)
# dataloader = DataLoader(WikiTextDataset(data[:40000]), batch_size=8, shuffle=True)
dataloader = DataLoader(WikiTextDataset(data[:min(len(data), 20000)]), batch_size=8, shuffle=True)

In [42]:
# 4) Transformer 기반 SLLM 모델 정의
# - [단어 인덱스] -> Token Embedding + Positional Embedding -> Transformer Encoder -> Linear(fc_out) -> [단어 확률 분포]
class TransformerSLLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, max_len):
        super().__init__()

        # 단어 인덱스를 고정된 차원의 벡터로 변환하는 임베딩 레이어를 정의 vocab_size(단어 사전의 크기), embed_dim(각 단어를 표현할 벡터의 차원, 예시 128차원)
        # 즉 단어 인덱스는 의미를 담고 있지 않음 그래서 임베딩 벡터는 의미적 유사성을 학습, 이 레이어는 vocab_size x embed_dim 크기의 임베딩 행렬을 학습한다
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)

        # 위지 정보(positional encoding) 입력 시쿼스의 위치 정보를 따로 제공해야 하며, 각 위치에 대해 학습 가능한 위치 벡터를 생성하는 임베딩 레이어
        # max_len(32), embed_dim(128) 위치 정보를 명시적으로 추가해야 문맥을 이해할 수 있음
        self.pos_embedding = nn.Embedding(max_len, embed_dim)

        # d_model(128 입력 및 출력 임베딩 차원), nhead(4 어텐셔 헤드의 수), dim_feedforward(256 내부의 은닉층 크기)
        # 즉 이 레이어는 입력 텐서의 shape이 [seq_len, batch_size, embed_dim]일때 동일한 shape의 출력을 반환하면서 문맥 정보를 강화한다
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)

        # Transformer 모델의 인코더 블록 전체를 구성하는 핵심 코드 앞서 정의한 encoder_layer를 여러번 반복해서 깊이 있는 문맥 이해를 가능하게 하는 구조
        # nn.TransformerEncoder 는 하나의 encoder_layer 를 num_layers 만큼 반복하여 전체 인코더 스택을 구성한다
        # Input → EncoderLayer 1 → EncoderLayer 2 → ... → EncoderLayer N → Output
        # 각 레이어는 입력 시퀀시를 받아 문맥 정보를 강화하고, 다음 레이어로 전달한다(반복 횟수가 많을수록 복잡한 문맥과 의미 관계를 더 잘 학습할 수 있음)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 모델의 출력층 정의하는 부분으로 모델이 예측한 임베딩을 단어 사전의 확률 분포로 변환하는 역할, Transformer의 마지막 출력은 [batch_size, seq_len, embed_dim] 형태의 문맥 벡터이다
        # 이 벡터를 vocab_size 차원의 로짓(logits)으로 변환하여, 각 위치에서 어떤 단어가 올지 에측한다, Linear 레이어는 임베딩 공간 -> 단어 공간으로 매핑하는 분류기 역할을 한다
        # embed_dim(Transformer의 출력 차원 예시 128), vocab_size(전체 단어 사전 크기 에시 10000)
        self.fc_out = nn.Linear(embed_dim, vocab_size)
        
    def forward(self, x):
        # x는 [batch_size, sequence_length] 형태의 텐서, 예를 들어, x.shape = [8, 32]라면:
        # x.size(0) → 배치 크기 (8)
        # x.size(1) → 시퀀스 길이 (32)
        # 즉, seq_len = 32가 된다
        seq_len = x.size(1)

        # seq_len = 32이면 [0, 1, 2, ..., 31] 형태의 위치 인덱스 벡터를 생성
        # 텐서의 shape을 [1, seq_len]으로 변환 → 이렇게 하면 배치 차원과 맞춰서 broadcasting이 가능
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)

        # 단어 임베딩과 위치 임베딩을 더해서 최종 입력 벡터를 만들고, 이 합산은 Transformer가 단어 의미 + 위치 정보를 동시에 인식할 수 있게 해준다
        x = self.token_embedding(x) + self.pos_embedding(positions) # 예시 [1, 32, 128]

        # Transformer 인코더에 입력을 전달하여 문맥 정보를 강화
        x = self.transformer(x)

        # Transformer의 출력은 [batch_size, seq_len, embed_dim] 형태의 문맥 벡터이며, fc_out 레이어를 통해 각 위치에서 다음에 올 단어의 확률 분포를 예측
        return self.fc_out(x)

In [43]:
# 5) 학습
model = TransformerSLLM(
    vocab_size=len(vocab),
    embed_dim=128,
    num_heads=4,
    hidden_dim=256,
    num_layers=2,
    max_len=sequence_length
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs) # 모델 예측
        loss = criterion(outputs.view(-1, len(vocab)), targets.view(-1)) # 손실함수 값

        # 오차역전파
        optimizer.zero_grad() # 미분 파리미터 초기화
        loss.backward() # 미분 연산
        optimizer.step() # 미분 연산 후 가중치,바이어스 파라미터 업데이트

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1, Loss: 9453.7484
Epoch 2, Loss: 6943.9914
Epoch 3, Loss: 6788.9130
Epoch 4, Loss: 6709.0129
Epoch 5, Loss: 6656.0690


In [ ]:
# 6) 텍스트 생성 함수 추가
# - 이 함수는 start_text를 기반으로 최대 50개의 단어를 이어서 생성합니다. 단어 사전에 없는 경우는 0으로 처리되며, 
# 학습된 문맥을 따라 다음 단어를 예측합니다.
# 샘플링 방식 추가 (top-k, top-p, temperature) - temperature=0.8 정도로 설정하면 더 다양하고 자연스러운 문장이 나올수 있음
# def generate_text(model, start_text, word2idx, idx2word, max_length=50, temperature=0.7):
def generate_text(model, start_text, word2idx, idx2word, max_length=50, temperature=0.7, top_k=50, top_p=0.9):
    model.eval()
    tokens = start_text.lower().split() # 공백 기준으로 단어 단위 분리 후 소문자 변환
    input_ids = [ word2idx.get(word, 0) for word in tokens ] # 단어 -> 숫자 인덱스 매핑 딕셔너리, 해당 단어가 없으면 0을 반환, unknown word -> 0

    # 텍스트 생성 함수에서 모델 입력을 위한 텐서를 준비하는 핵심 단계
    # 리스트 형태의 단어 인덱스 (input_ids)를 PyTorch 텐서로 변환, dtype=torch.long은 정수형 텐서로 지정 → nn.Embedding에서 필수
    # .unsqueeze(0) 텐서에 배치 차원(batch dimension)을 추가, 결과 shape: [1, seq_len] → 모델은 항상 [batch_size, seq_len] 형태의 입력을 기대함
    # 예시) input_ids = [12, 3, 45]
    # 예시) → tensor([12, 3, 45], dtype=torch.long)
    # 예시) → tensor([[12, 3, 45]])  # unsqueeze(0)
    # 예시) → tensor([[12, 3, 45]]).to(device)  # GPU 또는 CPU로 이동
    # 이렇게 준비된 input_tensor는 모델의 forward() 함수에 바로 넣을 수 있음
    input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)

    for _ in range(max_length):
        with torch.no_grad():
            output = model(input_tensor) # 모델 예측값
            # 마지막 토큰 위치의 출력, 출력 shape: [batch_size, seq_len, vocab_size]
            # temperature로 나누면 확률 분포의 sharpness를 조절합니다:
            # 낮은 값 (예: 0.5) → 더 결정적인 예측 (argmax에 가까움), 높은 값 (예: 1.0 이상) → 더 다양하고 창의적인 예측
            next_token_logits = output[:, -1, :] / temperature
            logits = top_k_filtering(next_token_logits, top_k)
            logits = top_p_filtering(logits, top_p)
            probabilities = torch.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probabilities, num_samples=1)
            # next_token_id = torch.argmax(next_token_logits, dim=-1).item() # argmax 방식으로 가장 확률 높은 단어만 선택
            input_ids.append(next_token_id)

            input_tensor = torch.tensor(input_ids[-sequence_length:], dtype=torch.long).unsqueeze(0).to(device)

    # 텐서를 정수로 변환 후 단어로 매핑
    generated_words = [idx2word.get(int(idx), "[UNK]") for idx in input_ids]
    return ' '.join(generated_words)

# 현재는 torch.multinomial()로 확률 기반 샘플링을 하고 있지만, 더 정교한 제어를 위해 top-k 또는 top-p 필터링을 추가
# 텍스트 생성 시 top-k 샘플링을 구현한 것으로, 모델이 예측한 단어들 중에서 가장 확률이 높은 상위 k개만 남기고 나머지는 제거하는 역할
# 모델이 예측한 단어 확률 분포(logits)에서 상위 top_k개만 남기고, 나머지 단어들은 선택되지 않도록 확률을 0으로 만드는 것, 이렇게 하면 의미 없는 희귀 단어가 선택되는 것을 방지하고, 더 자연스럽고 품질 높은 문장 생성이 가능
def top_k_filtering(logits, top_k=50):
    values, _ = torch.topk(logits, top_k) # torch.topk()는 가장 높은 top_k개의 값을 추출, logits는 [batch_size, vocab_size] 형태의 텐서
    # 1번 차원(두 번째 위치)에 새로운 차원을 추가, 즉, [1] → [1, 1]으로 바뀌는 것
    # 예시 values = tensor([[0.9, 0.3, 0.2]])  # shape: [1, 3]
    # 예시 values[:, -1] → tensor([0.2])       # shape: [1]
    # 예시 values[:, -1].unsqueeze(1) → tensor([[0.2]])  # shape: [1, 1]
    min_value = values[:, -1].unsqueeze(1) # values[:, -1]는 그 중 가장 낮은 값 (즉, top-k 중 가장 작은 로짓)을 의미, .unsqueeze(1)은 shape을 [batch_size, 1]로 만들어 broadcasting이 가능
    # logits 중에서 min_value보다 작은 값은 모두 -inf로 변환, -inf는 softmax 계산 시 확률이 0에 수렴하게 만들어 선택되지 않도록 한다, 최종적으로 남는 것은 top-k 단어들의 로짓만 유지된 텐서
    # 예시 logits = [0.1, 0.3, 0.05, 0.9, 0.2]
    # 예시 top_k = 3 → 유지: [0.3, 0.9, 0.2], 제거: [0.1, 0.05]
    # 예시 → 결과: [-inf, 0.3, -inf, 0.9, 0.2]
    return torch.where(logits < min_value, torch.full_like(logits, float('-inf')), logits)

# top-p 샘플링 추가 (nucleus sampling) - 모델이 예측한 단어 확률 분포(logits)에서 누적 확률이 top_p를 넘는 단어들을 제거하여, 자연스럽고 다양성 있는 문장 생성을 가능하게 함
def top_p_filtering(logits, top_p=0.9):
    # 로짓을 확률이 높은 순서대로 정렬, sorted_logits: 값만 정렬된 텐서, sorted_indices: 원래 위치를 기억하는 인덱스
    # 예시 logits = [0.1, 0.3, 0.05, 0.9, 0.2]
    # 예시 → sorted_logits = [0.9, 0.3, 0.2, 0.1, 0.05]
    # 예시 → sorted_indices = [3, 1, 4, 0, 2]
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)

    # 정렬된 로짓을 softmax로 확률화한 뒤, 누적 확률(cumulative sum)을 계산
    # 예시 softmax → [0.5, 0.2, 0.15, 0.1, 0.05]
    # 예시 cumulative_probs → [0.5, 0.7, 0.85, 0.95, 1.0]
    cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)

    # 누적 확률이 top_p(예: 0.9)를 초과하는 위치를 True로 표시
    # 예시 top_p = 0.9
    # 예시 → sorted_indices_to_remove = [False, False, False, True, True]
    sorted_indices_to_remove = cumulative_probs > top_p

    # 첫 번째 True부터 제거되도록 한 칸씩 밀어준다, 이렇게 하면 top_p를 넘는 첫 번째 단어는 유지하고, 그 이후부터 제거
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()

    # 가장 높은 확률을 가진 단어는 무조건 유지
    sorted_indices_to_remove[..., 0] = 0

    # 제거할 단어들의 원래 인덱스를 추출
    indices_to_remove = sorted_indices[sorted_indices_to_remove]

    # 제거 대상 로짓을 -inf로 설정 → softmax 시 확률이 0이 되어 선택되지 않음
    # logits = [0.1, 0.3, 0.05, 0.9, 0.2]
    # 예시 top_p = 0.9
    # 예시 → 유지: [0.9, 0.3, 0.2]
    # 예시 → 제거: [0.1, 0.05]
    # 예시 → 결과 logits = [-inf, 0.3, -inf, 0.9, 0.2]
    logits[0, indices_to_remove] = float('-inf')
    return logits

In [40]:
# 7) 테스트 실행 예시

# def clean_text(text):
#     text = text.replace(" .", ".").replace(" ,", ",").replace(" '", "'")
#     text = text.replace('" "', '"').replace("  ", " ")
#     text = text.replace("= = = = = = =", "")  # 반복 기호 제거
#     return text.strip()

import re
def clean_text(text):
    text = re.sub(r"(= ){3,}", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'"\s*,', ',', text)
    text = re.sub(r'\s*"\s*', '', text)
    text = re.sub(r'\s*\.\s*', '. ', text)
    return text.strip()

start_text = "Write a sentimental blog post about spring travel in Seoul. Mention cherry blossoms, street food, and quiet alleys."

generated = generate_text(model, start_text, word2idx, idx2word, max_length=50)
# print('Generated Text:\n', generated)
print("Generated Text:\n", clean_text(generated))

Generated Text:
 write a sentimental blog post about spring travel in ! mention cherry ! street ! and quiet !. the tower building , and its predecessors , the tower building was also known as they also no val ##ky ##ria chronicles iii outside japan , while maintaining a large team of the original scenario was assembled from the tower building is a field and the building was the
